## Ejercicios de pair programming 23 enero: Anova

In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd
import random 

# Estadísticos
# -----------------------------------------------------------------------
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv("../datos/world_risk_index_sin_outliers_est.csv", index_col = 0)
df.head(2)

,region,exposure_category,wri_category,vulnerability_category,susceptibility_category,wri,exposure,vulnerability,susceptibility,lack_of_coping_capabilities,lack_of_adaptive_capacities,year,exposure_Sklearn
11,Papua-Neuguinea,Very High,Very High,Very High,Very High,1.743241,23.26,1.047752,1.115836,0.774339,1.054301,2011.0,0.713429
12,Madagaskar,Very High,Very High,Very High,Very High,1.721174,20.68,0.633167,0.412299,0.861995,0.513743,2011.0,0.784972


In [3]:
outliers = pd.read_csv("../datos/world_risk_index_outliers_est.csv", index_col = 0)
outliers.head(2)

,region,exposure_category,wri_category,vulnerability_category,susceptibility_category,wri,exposure,vulnerability,susceptibility,lack_of_coping_capabilities,lack_of_adaptive_capacities,year,exposure_Sklearn
0,Vanuatu,Very High,Very High,High,High,1.640675,56.33,0.801253,0.792708,0.541556,0.926242,2011.0,0.563758
1,Tonga,Very High,Very High,Medium,Medium,1.292570,56.04,0.376459,0.030528,0.707655,0.185736,2011.0,0.560853


### Info columnas
|Columna| Tipo de dato | Descripcion |
|-------|--------------|-------------|
|Region| String|	Name of the region.
|WRI	| Decimal |	World Risk Score of the region.
|Exposure	| Decimal |	Risk/exposure to natural hazards such as earthquakes, hurricanes, floods, droughts, and sea ​​level rise.
|Vulnerability	| Decimal |	Vulnerability depending on infrastructure, nutrition, housing situation, and economic framework conditions.
|Susceptibility	| Decimal |	Susceptibility depending on infrastructure, nutrition, housing situation, and economic framework conditions.
|Lack of Coping Capabilities	| Decimal |	Coping capacities in dependence of governance, preparedness and early warning, medical care, and social and material security.
|Lack of Adaptive Capacities| Decimal |	Adaptive capacities related to coming natural events, climate change, and other challenges.
|Year	| Decimal |	Year data is being described.
|WRI Category| String|	WRI Category for the given WRI Score.
|Exposure Category| String|	Exposure Category for the given Exposure Score.
|Vulnerability Categoy| String|	Vulnerability Category for the given Vulnerability Score.
|Susceptibility Category| String|	Susceptibility Category for the given Susceptibility Score.

Link a la base de datos : https://www.kaggle.com/datasets/tr1gg3rtrash/global-disaster-risk-index-time-series-dataset

### Nuestra variable respuesta es Exposure_Sklearn, queremos saber cual es el riesgo de desastres naturales dependiendo del resto de variables

In [4]:
df.columns

Index(['region', 'exposure_category', 'wri_category', 'vulnerability_category',
       'susceptibility_category', 'wri', 'exposure', 'vulnerability',
       'susceptibility', 'lack_of_coping_capabilities',
       'lack_of_adaptive_capacities', 'year', 'exposure_Sklearn'],
      dtype='object')

In [5]:
lm = ols("exposure_Sklearn ~ region  + exposure_category + wri_category + vulnerability_category + susceptibility_category + wri + vulnerability + susceptibility + lack_of_coping_capabilities + lack_of_adaptive_capacities + year", data=df).fit()
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
region,282.0,8.374185,0.029696,16.268289,7.719912e-272
exposure_category,4.0,0.083940,0.020985,11.496338,3.663632e-09
wri_category,4.0,0.328683,0.082171,45.015811,2.798619e-35
vulnerability_category,4.0,0.116761,0.029190,15.991359,9.143612e-13
susceptibility_category,4.0,0.268068,0.067017,36.714101,5.290672e-29
wri,1.0,28.092702,28.092702,15390.118113,0.000000e+00
vulnerability,1.0,10.415727,10.415727,5706.082342,0.000000e+00
susceptibility,1.0,0.025191,0.025191,13.800276,2.125700e-04
lack_of_coping_capabilities,1.0,0.076830,0.076830,42.090285,1.267312e-10
lack_of_adaptive_capacities,1.0,0.000136,0.000136,0.074650,7.847297e-01


El DF nos indica las que son columnas categorica (*region, exposure_category, wri_category,vulnerability_category, susceptibility_category*) y numerica todas las que tienen un valor de 1.

El F evalua la capacidad que tiene cada variable predictora de influir sobre la variable respuesta. Por lo cual la que influyen mas son *wri* y *vulnerability*

Mirando la columna del PR(>F) podemos concluir que *lack_of_adaptive_capacities y year* son mayores de 0.05 por lo cual NO influyen sobre nuestra variable respuesta.

In [6]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       exposure_Sklearn   R-squared:                       0.956
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     86.97
Date:                Mon, 23 Jan 2023   Prob (F-statistic):               0.00
Time:                        12:48:22   Log-Likelihood:                 2792.0
No. Observations:                1513   AIC:                            -4980.
Df Residuals:                    1211   BIC:                            -3373.
Df Model:                         301                                         
Covariance Type:            nonrobust                                         
==============================================================================================================
                                                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
Intercept                                      1.1714      0.701      1.670      0.095      -0.205       2.547
region[T.Albania]                              0.0329      0.050      0.654      0.513      -0.066       0.132
region[T.Albanien]                             0.0209      0.030      0.687      0.492      -0.039       0.080
region[T.Algeria]                             -0.0087      0.049     -0.175      0.861      -0.106       0.088
region[T.Algerien]                            -0.0066      0.029     -0.231      0.817      -0.063       0.050
region[T.Angola]                              -0.0017      0.020     -0.085      0.932      -0.041       0.037
region[T.Argentina]                           -0.0122      0.051     -0.240      0.811      -0.112       0.088
region[T.Argentinien]                         -0.0333      0.031     -1.075      0.283      -0.094       0.028
region[T.Armenia]                             -0.0538      0.051     -1.051      0.294      -0.154       0.047
region[T.Armenien]                             0.0265      0.030      0.881      0.379      -0.033       0.086
region[T.Aserbaidschan]                        0.0018      0.029      0.062      0.951      -0.055       0.059
region[T.Australia]                            0.0067      0.053      0.126      0.900      -0.098       0.111
region[T.Australien]                           0.0301      0.035      0.866      0.387      -0.038       0.098
region[T.Austria]                              0.0128      0.053      0.243      0.808      -0.090       0.116
region[T.Azerbaijan]                          -0.1140      0.050     -2.297      0.022      -0.211      -0.017
region[T.Bahamas]                             -0.0249      0.031     -0.810      0.418      -0.085       0.035
region[T.Bahrain]                             -0.0051      0.033     -0.154      0.878      -0.071       0.060
region[T.Barbados]                            -0.0070      0.033     -0.212      0.832      -0.071       0.057
region[T.Belarus]                             -0.0094      0.038     -0.245      0.806      -0.085       0.066
region[T.Belgien]                             -0.0056      0.035     -0.160      0.873      -0.074       0.063
region[T.Belgium]                             -0.0067      0.053     -0.125      0.900      -0.111       0.098
region[T.Belize]                              -0.0068      0.028     -0.249      0.804      -0.061       0.047
region[T.Benin]                                0.0041      0.020      0.199      0.843      -0.036       0.044
region[T.Bhutan]                              -0.0080      0.025     -0.321      0.748      -0.057       0.041
region[T.Bolivia]                             -0.0937      0.048     -1.933      0.053

- El P>|t| de la columna *region* en algunas regiones es menor de 0.05 por lo cual nos quedamos con esta columna (la region influye sobre la variable respuesta).
- exposure_category también influye.  
- El P>|t| se ve afectado especialmente por la categoria de region al ser que tenemos unas 200 mas o menos.
- R square de nuestra variables predictoras explican un 95% de nuestra variable respuesta. 

In [9]:
outliers.head()

,region,exposure_category,wri_category,vulnerability_category,susceptibility_category,wri,exposure,vulnerability,susceptibility,lack_of_coping_capabilities,lack_of_adaptive_capacities,year,exposure_Sklearn
0,Vanuatu,Very High,Very High,High,High,1.640675,56.33,0.801253,0.792708,0.541556,0.926242,2011.0,0.563758
1,Tonga,Very High,Very High,Medium,Medium,1.292570,56.04,0.376459,0.030528,0.707655,0.185736,2011.0,0.560853
2,Philippinen,Very High,Very High,High,High,0.725110,45.09,0.552087,0.592868,0.773824,0.106661,2011.0,0.451167
3,Salomonen,Very High,Very High,Very High,High,0.628547,36.40,1.475212,1.440562,0.987862,1.731821,2011.0,0.364119
4,Guatemala,Very High,Very High,High,High,0.315014,38.42,0.588423,0.627259,0.439602,0.589349,2011.0,0.384353


In [11]:
outliers.isnull().sum()

region                           0
exposure_category                0
wri_category                     0
vulnerability_category           0
susceptibility_category          0
wri                            178
exposure                         0
vulnerability                  178
susceptibility                 178
lack_of_coping_capabilities    178
lack_of_adaptive_capacities    178
year                             0
exposure_Sklearn               178
dtype: int64

In [8]:
lm_outliers = ols("exposure_Sklearn ~ region  + exposure_category + wri_category + vulnerability_category + susceptibility_category + wri + vulnerability + susceptibility + lack_of_coping_capabilities + lack_of_adaptive_capacities + year", data=outliers).fit()
sm.stats.anova_lm(lm_outliers)

ValueError: shapes (12,53) and (33,) not aligned: 53 (dim 1) != 33 (dim 0)

Nos hemos dado cuenta que las columnas con la que nos quedamos son las misma del dataframe DF.

lm.summary()